In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to C:\Users\Arjun
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,id,comment_text,toxic
0,e617e2489abe9bca,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",0
1,9250cf637294e09d,"""\r\n\r\nThis seems unbalanced. whatever I ha...",0
2,ce1aa4592d5240ca,"Marya Dzmitruk was born in Minsk, Belarus in M...",0
3,48105766ff7f075b,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,0
4,0543d4f82e5470b6,New Categories \r\n\r\nI honestly think that w...,0


In [5]:
df["tokenized"] = df["comment_text"]
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x) 
df["tokenized"] = df['tokenized'].str.replace('[^\w\s]','')

C:\Users\ARJUNV~1\AppData\Local\Temp/ipykernel_26168/1293593572.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["tokenized"] = df['tokenized'].str.replace('[^\w\s]','')


In [6]:
df.head()

,id,comment_text,toxic,tokenized
0,e617e2489abe9bca,"""\r\n\r\n a barnstar for you! \r\n\r\n the de...",0,\r\n\r\n a barnstar for you \r\n\r\n the defe...
1,9250cf637294e09d,"""\r\n\r\nthis seems unbalanced. whatever i ha...",0,\r\n\r\nthis seems unbalanced whatever i have...
2,ce1aa4592d5240ca,"marya dzmitruk was born in minsk, belarus in m...",0,marya dzmitruk was born in minsk belarus in ma...
3,48105766ff7f075b,"""\r\n\r\ntalkback\r\n\r\n dear celestia... """,0,\r\n\r\ntalkback\r\n\r\n dear celestia
4,0543d4f82e5470b6,new categories \r\n\r\ni honestly think that w...,0,new categories \r\n\r\ni honestly think that w...


In [7]:
df["tokenized"] = df["tokenized"].apply(word_tokenize)
df["tokenized"].head()

0    [a, barnstar, for, you, the, defender, of, the...
1    [this, seems, unbalanced, whatever, i, have, s...
2    [marya, dzmitruk, was, born, in, minsk, belaru...
3                           [talkback, dear, celestia]
4    [new, categories, i, honestly, think, that, we...
Name: tokenized, dtype: object

In [8]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df["tokenized"] = df["tokenized"].apply(lambda words: [word for word in words if word not in stop_words])

[nltk_data] Downloading package stopwords to C:\Users\Arjun
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
stemmer = PorterStemmer()
df['stemmed'] = df['tokenized'].apply(lambda x: [stemmer.stem(y) for y in x])

In [10]:
my_dict = pd.Series(df.stemmed.values,index=df.id).to_dict()


In [11]:
my_list = df["stemmed"]
my_list[:5]

0    [barnstar, defend, wiki, barnstar, like, edit,...
1    [seem, unbalanc, whatev, said, mathsci, said, ...
2    [marya, dzmitruk, born, minsk, belaru, march, ...
3                           [talkback, dear, celestia]
4    [new, categori, honestli, think, need, add, ca...
Name: stemmed, dtype: object

In [12]:
words = []
for key in my_dict:
    for i in range(len(my_dict[key])):
        term = my_dict[key][i]
        words.append(term)

In [13]:
length = len(my_list)
inverted_index = {}
for i in range(length):
    check = my_list[i]
    for item in words:
        if item in check:
            if item not in inverted_index:
                inverted_index[item] = []
            if item in inverted_index:
                inverted_index[item].append(i+1)

In [14]:
for key in inverted_index:
    inverted_index[key] = list(set(inverted_index[key]))
inverted_index["minsk"]

[3]

In [15]:
def boolean_query(query):
  query = query.lower()

  args = list(query.split(' '))

  bool_q = ['and', 'or', 'not']
  q_size = len(args)

  
  wildcard=[]

  
  for arg in args:
    if "*" in arg:
      wildcard.append(arg)
  
  
  for i in range(len(args)):
    if (args[i] not in bool_q) and (args[i] not in words) and (args[i] not in wildcard):
      args[i]=spell_correction(words,args[i])

  if args[0]=='not' and (q_size!=1) and (args[1] not in bool_q) and (args[1] not in wildcard):
    ans = find_not(inverted_index[args[1]])
    i=2
  elif args[0]=='not' and q_size!=1 and (args[1] not in bool_q) and (args[1] in wildcard):
    ans=find_not(wildcard_query(args[1]))
    i=2
  elif (args[0] not in bool_q) and (args[0] not in wildcard):
    ans = inverted_index[args[0]]
    i=1
  elif (args[0] not in bool_q) and (args[0] in wildcard):
    ans=wildcard_query(args[0])
    i=1
  else:
    print("Invalid Query")
    return []
  
  while i<q_size-1:
    if args[i]=='not' and (args[i+1] not in bool_q) and (args[i+1] not in wildcard):
      ans = find_not(inverted_index[args[i+1]])
      i+=2
    elif args[i]=='not' and (args[i+1] not in bool_q) and (args[i+1] in wildcard):
      ans=find_not(wildcard_query(args[i+1]))
      i+=2
    elif args[i]=='and' and args[i+1]!='not' and (args[i+1] not in wildcard):
      ans = find_and(ans,inverted_index[args[i+1]])
      i+=2
    elif args[i]=='and' and args[i+1]!='not' and (args[i+1] in wildcard):
      ans=find_and(ans,wildcard_query(args[i+1]))
      i+=2
    elif args[i]=='and' and (i<q_size-2) and (args[i+2] not in wildcard):
      list1 = find_not(inverted_index[args[i+2]])
      ans = find_and(ans,list1)
      i+=3
    elif args[i]=='and' and (i<q_size-2) and (args[i+2] in wildcard):
      list1=find_not(wildcard_query(args[i+2]))
      ans = find_and(ans,list1)
      i+=3
    elif args[i]=='or' and (args[i+1]!='not') and (args[i+1] not in wildcard):
      ans = find_or(ans, inverted_index[args[i+1]])
      i+=2
    elif args[i]=='or' and (args[i+1]!='not') and (args[i+1] in wildcard):
      ans= find_or(ans, wildcard_query(args[i+1]))
      i+=2
    elif args[i]=='or' and (i<q_size-2) and (args[i+2] not in wildcard):
      list1 = find_not(inverted_index[args[i+2]])
      ans = find_or(ans,list1)
      i+=3
    elif args[i]=='or' and (i<q_size-2) and (args[i+2] in wildcard):
      list1=find_not(wildcard_query(args[i+2]))
      ans=find_or(ans,list1)
      i+=3
    else:
      print("Invalid Query")
      ans = []
      break
  return ans

In [16]:
def prefix_match(query):
  res=[]
  for token in permuterm_index:
    if token.startswith(query):
      res.append(permuterm_index[token])
  final_res=set(res)
  return final_res

In [17]:
def rotate_query(query):
  query+='$'
  length=len(query)
  for d in range(length):
    Rfirst = query[0 : len(query)-d]
    Rsecond = query[len(query)-d : ]
    rotated_query=Rsecond+Rfirst
    if rotated_query[-1]=='*':
      res=rotated_query
      break
  return res[:-1]

def spell_correction(tokens,word):
  edit_dist=10000000
  for token in tokens:
    dist=nltk.edit_distance(token,word)
    if dist<edit_dist:
      ans=token
      edit_dist=dist
    elif dist==edit_dist:
      if len(inverted_index[token])>len(inverted_index[ans]):
        ans=token
  return ans

In [18]:
def find_and(list1,list2):
  result=list(set(list1) & set(list2))
  return result

def find_not(list1):
  result=list(set(universal) - set(list1))
  return result

def find_or(list1,list2):
  list1.extend(list2)
  result=list(set(list1))
  return result

def permute(input):
    tmp=input
    s=len(input)+1
    input+='$'+input
    res=[]
    for x in range(s):
        res.append(input[x:x+s])
    return res

In [19]:
permuterm_index={}

def permuterm(tokens):
  for token in tokens:
    res=permute(token)
    for item in res:
      permuterm_index[item]=token

permuterm(words)

print(len(permuterm_index))


162279


In [62]:
user_query=input("Enter your query:")
results=boolean_query(user_query)

Enter your query:barnster and defender


In [63]:
results

[2392, 1]